In [48]:
import pandas
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.onnx
import numpy

In [28]:
# Load the dataset into a dataframe
dataframe = pandas.read_csv("../datasets/maternal_health_risk.csv")

In [29]:
dataframe = pandas.get_dummies(dataframe, columns=['RiskLevel'])

In [30]:
print(dataframe.iloc[[0],:6]) # inputs

   Age  SystolicBP  DiastolicBP    BS  BodyTemp  HeartRate
0   25         130           80  15.0      98.0         86


In [31]:
print(dataframe.iloc[[0],6:]) # inputs

   RiskLevel_high risk  RiskLevel_low risk  RiskLevel_mid risk
0                 True               False               False


In [32]:
# split dataset to train (70%) and test (30%)
train, test = numpy.split(dataframe.to_numpy(dtype="float32"), [int(.7*len(dataframe))])

In [33]:
# further split training and testing to their corresponding inputs and outputs
train_torch = torch.from_numpy(train)
test_torch = torch.from_numpy(test)

train_x = train_torch[:,:6]
train_y = train_torch[:,6:]
test_x = test_torch[:,:6]
test_y = test_torch[:,6:]

In [34]:
print(train_x[0])
print(train_y[0])

tensor([ 25., 130.,  80.,  15.,  98.,  86.])
tensor([1., 0., 0.])


In [35]:
model = nn.Sequential(
    nn.Linear(6, 12),
    nn.Linear(12, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.Linear(12, 6),
    nn.Linear(6, 3)
)

In [36]:
# loss function and optimizer
loss_function = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [37]:
# training parameters
n_epochs = 20000   # number of epochs to run

In [38]:
for index in range(n_epochs):
    prediction_y = model(train_x)
    step_loss = loss_function(prediction_y, train_y)
    optimizer.zero_grad()
    step_loss.backward()
    optimizer.step()
    print ('epoch [{}], Loss: {:.2f}'.format(index, step_loss.item()))

epoch [0], Loss: 22.47
epoch [1], Loss: 22.42
epoch [2], Loss: 22.37
epoch [3], Loss: 22.32
epoch [4], Loss: 22.27
epoch [5], Loss: 22.22
epoch [6], Loss: 22.17
epoch [7], Loss: 22.12
epoch [8], Loss: 22.08
epoch [9], Loss: 22.03
epoch [10], Loss: 21.98
epoch [11], Loss: 21.93
epoch [12], Loss: 21.88
epoch [13], Loss: 21.83
epoch [14], Loss: 21.78
epoch [15], Loss: 21.73
epoch [16], Loss: 21.69
epoch [17], Loss: 21.64
epoch [18], Loss: 21.59
epoch [19], Loss: 21.54
epoch [20], Loss: 21.49
epoch [21], Loss: 21.45
epoch [22], Loss: 21.40
epoch [23], Loss: 21.35
epoch [24], Loss: 21.30
epoch [25], Loss: 21.26
epoch [26], Loss: 21.21
epoch [27], Loss: 21.16
epoch [28], Loss: 21.11
epoch [29], Loss: 21.07
epoch [30], Loss: 21.02
epoch [31], Loss: 20.97
epoch [32], Loss: 20.93
epoch [33], Loss: 20.88
epoch [34], Loss: 20.83
epoch [35], Loss: 20.79
epoch [36], Loss: 20.74
epoch [37], Loss: 20.69
epoch [38], Loss: 20.65
epoch [39], Loss: 20.60
epoch [40], Loss: 20.56
epoch [41], Loss: 20.51
ep

In [39]:
prediction_y = model(test_x)
print(prediction_y)
print(test_y)

tensor([[ 0.0429,  0.5765,  0.4117],
        [ 0.5412,  0.2697,  0.1888],
        [ 0.0991,  0.5868,  0.3591],
        [ 0.0736,  0.6051,  0.3614],
        [ 0.1130,  0.6570,  0.2515],
        [ 0.0596,  0.7151,  0.2215],
        [ 0.1265,  0.6341,  0.2483],
        [ 0.0589,  0.6160,  0.3639],
        [ 0.2159,  0.5736,  0.2992],
        [ 0.0594,  0.4833,  0.4998],
        [ 0.0692,  0.4849,  0.4434],
        [ 0.0664,  0.5802,  0.3397],
        [ 0.3696,  0.5040,  0.1806],
        [ 0.2368,  0.3932,  0.4107],
        [ 0.1916,  0.6181,  0.2506],
        [ 0.0594,  0.4833,  0.4998],
        [ 0.1094,  0.4538,  0.4330],
        [ 0.3448,  0.5077,  0.2308],
        [ 0.0664,  0.5802,  0.3397],
        [ 0.0449,  0.5618,  0.4223],
        [ 0.0664,  0.5802,  0.3397],
        [ 0.0429,  0.5765,  0.4117],
        [ 0.5412,  0.2697,  0.1888],
        [ 0.0991,  0.5868,  0.3591],
        [ 0.0736,  0.6051,  0.3614],
        [ 0.1130,  0.6570,  0.2515],
        [ 0.0596,  0.7151,  0.2215],
 

In [44]:
# Accuracy when matching the interpretation of inputs and outputs
prediction_boolean = prediction_y.detach().numpy() > 0.5
test_boolean = test_y.numpy() > 0.5
correct_boolean = (prediction_boolean == test_boolean).sum()
total_boolean = test_boolean.size
print('Match: {}, Total: {}, Rate: {:.2f}%'.format(correct_boolean, total_boolean, correct_boolean/total_boolean * 100))

Match: 573, Total: 915, Rate: 62.62%


In [46]:
# Accuracy when EXACT label basis
prediction_labeled = numpy.argmax(prediction_y.detach().numpy(), axis=-1)
test_labeled = numpy.argmax(test_y.numpy(), axis=-1)
correct_answers = (prediction_labeled == test_labeled).sum()
total = test_labeled.size
print('Exact: {}, Total: {}, Rate: {:.2f}%'.format(correct_answers, total, correct_answers/total * 100))

Exact: 136, Total: 305, Rate: 44.59%


In [47]:
model.eval()
torch.onnx.export(model, test_x[0], "../onnx/maternal_health_risk_model.onnx", export_params=True)